In [164]:
import pandas as pd
import numpy as np
import seaborn as sb
import datetime
import os
import sys
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

In [165]:
case_names = [    
              "Thermal_Base",
              # "2_Hr_BESS", 
              # "2_Hr_BESS_Fuelx2",
              # "4_Hr_BESS",
              # "4_Hr_BESS_Fuelx2",
              # "4_Hr_BESS_Fuelx3",
              # "4_Hr_BESS_Fuelx4",
              # "6_Hr_BESS",
              # "6_Hr_BESS_Fuelx2",
            #   "8_Hr_BESS",
            #   "8_Hr_BESS_Fuelx2",
            #   "10_Hr_BESS",
            #   "10_Hr_BESS_Fuelx2",
              ]

In [166]:
import os

current_dir = os.getcwd()
print(current_dir)

c:\Users\ks885\Documents\aa_research\Modeling\spcm_genx_experiment\figures


In [167]:
plots_path = os.path.join(current_dir, 'plots') + "/"
tables_path = os.path.join(current_dir, 'tables') + "/"

In [168]:
# modeling scaling ModelScalingFactor
ModelScalingFactor = 1000

cem_path = os.path.join(os.path.dirname(current_dir), 'GenX.jl', 'research_systems')
policies_path = os.path.join(os.path.dirname(current_dir), 'SPCM', 'research_systems')

params_names = ['Inv_cost_MW', 'Inv_cost_MWh', 'Fixed_OM_cost_MW', 'Var_OM_cost_out', 
          'Fuel_cost', 'Var_OM_cost_in', 'StartCost', 'Charge_cost', 'EnergyRevenue', 
          'OperatingReserveRevenue', 'OperatingRegulationRevenue',]

param_print_names = params_names

for i in range(len(param_print_names)):
    param_print_names[i] = param_print_names[i].replace("_", " ")

cost_params = ['Inv_cost_MW', 'Inv_cost_MWh', 'Fixed_OM_cost_MW', 'Var_OM_cost_out', 
        'Fuel_cost', 'Var_OM_cost_in', 'StartCost', 'Charge_cost']

revenue_params = ['EnergyRevenue', 'OperatingReserveRevenue', 'OperatingRegulationRevenue']

In [169]:
# get load data

load_data_raw = pd.read_csv(cem_path + '\\Thermal_Base\\' + 'system\\' + 'Demand_data.csv')
load_data = load_data_raw['Demand_MW_z1']

In [170]:
date = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

filename = 'operational_profits_' + date + '.pdf'

In [171]:
unit_profit_dic = {case: None for case in case_names}
percentage_profits_dic = {case: None for case in case_names}
consumer_payments_dic = {case: None for case in case_names}
total_investment_costs_dic = {case: None for case in case_names}
total_operating_costs_dic = {case: None for case in case_names}
total_system_costs_dic = {case: None for case in case_names} 
total_inv_op_costs_dic = {case: None for case in case_names}
total_objective_dic = {case: None for case in case_names}
nse_costs_dic = {case: None for case in case_names}
total_nse_dic = {case: None for case in case_names}
total_unmet_rsv_dic = {case: None for case in case_names}
total_nse_cost_dic = {case: None for case in case_names}
total_unmet_rsv_cost_dic = {case: None for case in case_names}
total_fixed_om_costs_dic = {case: None for case in case_names}
total_var_om_out_costs_dic = {case: None for case in case_names}
total_fuel_costs_dic = {case: None for case in case_names}
total_var_om_in_costs_dic = {case: None for case in case_names}
total_start_costs_dic = {case: None for case in case_names}
total_charge_costs_dic = {case: None for case in case_names}

In [172]:
vre_dfGen = pd.read_csv(policies_path + '\\' + case_names[0] + '\\resources' + '\\Vre.csv')

In [173]:
vre_dfGen = pd.read_csv(policies_path + '\\' + case_names[0] + '\\resources' + '\\Vre.csv')
thermal_dfGen = pd.read_csv(policies_path + '\\' + case_names[0] + '\\resources' + '\\Thermal.csv')
storage_dfGen = pd.read_csv(policies_path + '\\' + case_names[0] + '\\resources' + '\\Storage.csv')

In [174]:
dfGen = pd.concat([vre_dfGen, thermal_dfGen, storage_dfGen], ignore_index=True)

In [175]:
dfGen

,Resource,Zone,Num_VRE_Bins,New_Build,Can_Retire,Existing_Cap_MW,Max_Cap_MW,Min_Cap_MW,Inv_Cost_per_MWyr,Fixed_OM_Cost_per_MWyr,...,Max_Cap_MWh,Min_Cap_MWh,Inv_Cost_per_MWhyr,Fixed_OM_Cost_per_MWhyr,Var_OM_Cost_per_MWh_In,Self_Disch,Eff_Up,Eff_Down,Min_Duration,Max_Duration
0,Land-Based Wind - Class 1 - Technology 1,1,1.0,1,0,32791.061172,-1,0,83289.329169,32443.047267,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Utility PV - Class 1,1,1.0,1,0,17972.934821,-1,0,74134.164015,23765.603456,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NG 2-on-1 Combined Cycle (F-Frame),1,NaN,1,0,46274.023418,-1,0,56221.989154,25875.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NG Combustion Turbine (F-Frame),1,NaN,1,0,9457.159921,-1,0,50580.567617,19575.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [176]:
for case_name in case_names:

    print('Case Name: ' + case_name + '\n')

    # load generator characteristics from resources folder
    vre_dfGen = pd.read_csv(policies_path + '\\' + case_name + '\\resources' + '\\Vre.csv')
    thermal_dfGen = pd.read_csv(policies_path + '\\' + case_name + '\\resources' + '\\Thermal.csv')
    storage_dfGen = pd.read_csv(policies_path + '\\' + case_name + '\\resources' + '\\Storage.csv')
    # combine all resources to dfGen
    dfGen = pd.concat([vre_dfGen, thermal_dfGen, storage_dfGen], ignore_index=True)

    gen_capacity_gw = dfGen['Existing_Cap_MW'] / ModelScalingFactor
    gen_capacity_mw = dfGen['Existing_Cap_MW']
    gen_capacity_mwhour = dfGen['Existing_Cap_MWh']
    gen_capacity_mwhour = dfGen['Existing_Cap_MWh'] / ModelScalingFactor

    resource_list = dfGen['Resource']

    # solar and wind id
    solar_id = dfGen[dfGen['Resource'] == 'Utility PV - Class 1'].index[0]
    wind_id = dfGen[dfGen['Resource'] == 'Land-Based Wind - Class 1 - Technology 1'].index[0]

    cem_case = 'CEM_' + case_name + '_ABB\\'
    pf_case = 'PF_' + case_name + '\\'
    dlac_case = 'DLAC_' + case_name + '\\'
    dlac_imperfect_case = 'DLAC_imperfect_' + case_name + '\\'
    slac_case = 'SLAC_' + case_name + '\\'

    # define print names
    cem_case_print = cem_case.replace('_', ' ')
    cem_case_print = cem_case_print.replace('\\', '')
    cem_case_print = cem_case_print.replace('ABB', '')

    pf_case_print = pf_case.replace('_', ' ')
    pf_case_print = pf_case_print.replace('\\', '')

    dlac_case_print = dlac_case.replace('_', ' ')
    dlac_case_print = dlac_case_print.replace('\\', '')

    dlac_imperfect_case_print = dlac_imperfect_case.replace('_', ' ')
    dlac_imperfect_case_print = dlac_imperfect_case_print.replace('\\', '')

    slac_case_print = slac_case.replace('_', ' ')
    slac_case_print = slac_case_print.replace('\\', '')

    cem_results_path = cem_path + '\\' + case_name + '\\' + 'results\\'
    pf_results_path = policies_path + '\\' + case_name + '\\' + 'results_pf\\'
    dlac_results_path = policies_path + '\\' + case_name + '\\' + 'results_dlac-p\\'
    dlac_imperfect_results_path = policies_path + '\\' + case_name + '\\' + 'results_dlac-i\\'
    slac_results_path = policies_path + '\\' + case_name + '\\' + 'results_slac\\'

    cem_raw = pd.read_csv(cem_results_path + 'NetRevenue.csv')
    pf_raw = pd.read_csv(pf_results_path + 'NetRevenue.csv')
    dlac_raw = pd.read_csv(dlac_results_path + 'NetRevenue.csv')
    dlac_imperfect_raw = pd.read_csv(dlac_imperfect_results_path + 'NetRevenue.csv')
    slac_raw = pd.read_csv(slac_results_path + 'NetRevenue.csv')

    ### XXX Fix inv cost MWh
    # dlac_raw['Inv_cost_MWh'] = cem_raw['Inv_cost_MWh']
    # dlac_imperfect_raw['Inv_cost_MWh'] = cem_raw['Inv_cost_MWh']
    # slac_raw['Inv_cost_MWh'] = cem_raw['Inv_cost_MWh']


    cem_raw_prices = pd.read_csv(cem_results_path + 'prices.csv')
    pf_raw_prices = pd.read_csv(pf_results_path + 'price_electricity.csv', header=None)
    dlac_raw_prices = pd.read_csv(dlac_results_path + 'price_electricity.csv', header=None)
    dlac_imperfect_raw_prices = pd.read_csv(dlac_imperfect_results_path + 'price_electricity.csv', header=None)
    slac_raw_prices = pd.read_csv(slac_results_path + 'price_electricity.csv', header=None)


    column_names = cem_raw.columns[1:]
    resource_names = cem_raw['Resource'].unique()
    gen_print_names = resource_names
    print(gen_print_names)

    param_print_names = params_names

    for i in range(len(param_print_names)):
        param_print_names[i] = param_print_names[i].replace("_", " ")

    for i in range(len(resource_names)):
        gen_print_names[i] = gen_print_names[i].replace("Coal-new", "Coal")
        gen_print_names[i] = gen_print_names[i].replace("NG Combustion Turbine (F-Frame)", "NG CT")
        gen_print_names[i] = gen_print_names[i].replace("NG Combined Cycle (F-Frame)", "NG CC")
        gen_print_names[i] = gen_print_names[i].replace("Utility PV - Class 1", "Solar")
        gen_print_names[i] = gen_print_names[i].replace("Land-Based Wind - Class 1 - Technology 1", "Wind")
        gen_print_names[i] = gen_print_names[i].replace("Utility-Scale Battery Storage", "Battery")

    # get cem total nse and unmet rsv and total cem nse and unmet rsv costs
    cem_costs_genx = pd.read_csv(cem_results_path + 'costs.csv')
    cem_sum_ur_cost = cem_costs_genx[cem_costs_genx['Costs'] == 'cUnmetRsv']['Total'].values[0]
    cem_total_unmet_rsv = cem_sum_ur_cost / 1000
    cem_total_nse_cost = cem_costs_genx[cem_costs_genx['Costs'] == 'cNSE']['Total'].values[0]
    cem_total_nse = cem_total_nse_cost / 5000

    # load in timeseries of nse and unmet rsv from LACs
    cem_nse = pd.read_csv(cem_results_path + 'nse.csv')
    pf_nse = pd.read_csv(pf_results_path + 'zone_nse.csv', header=None)
    dlac_nse = pd.read_csv(dlac_results_path + 'zone_nse.csv', header=None)
    dlac_imperfect_nse = pd.read_csv(dlac_imperfect_results_path + 'zone_nse.csv', header=None)
    slac_nse = pd.read_csv(slac_results_path + 'zone_nse.csv', header=None)

    dlac_ur = pd.read_csv(dlac_results_path + 'zone_unmet_rsv.csv', header=None)
    pf_ur = pd.read_csv(pf_results_path + 'zone_unmet_rsv.csv', header=None)
    dlac_imperfect_ur = pd.read_csv(dlac_imperfect_results_path + 'zone_unmet_rsv.csv', header=None)
    slac_ur = pd.read_csv(slac_results_path + 'zone_unmet_rsv.csv', header=None)

    pf_nse_total = pf_nse.sum(axis=1)
    dlac_nse_total = dlac_nse.sum(axis=1)
    dlac_imperfect_nse_total = dlac_imperfect_nse.sum(axis=1)
    slac_nse_total = slac_nse.sum(axis=1)

    pf_ur_total = pf_ur.sum(axis=1)
    dlac_ur_total = dlac_ur.sum(axis=1)
    dlac_imperfect_ur_total = dlac_imperfect_ur.sum(axis=1)
    slac_ur_total = slac_ur.sum(axis=1)

    # print(dlac_nse_total)

    nse_totals = {
        'CEM': f"{cem_total_nse:.4e}",
        'PF': f"{pf_nse_total[0]:.4e}",
        'DLAC': f"{dlac_nse_total[0]:.4e}",
        'DLAC Imperfect': f"{dlac_imperfect_nse_total[0]:.4e}",
        'SLAC': f"{slac_nse_total[0]:.4e}"
    }

    unmet_rsv_totals = {
        'CEM': f"{cem_total_unmet_rsv:.4e}",
        'PF': f"{pf_ur_total[0]:.4e}",
        'DLAC': f"{dlac_ur_total[0]:.4e}",
        'DLAC Imperfect': f"{dlac_imperfect_ur_total[0]:.4e}",
        'SLAC': f"{slac_ur_total[0]:.4e}"
    }


    total_nse_dic[case_name] = nse_totals
    total_unmet_rsv_dic[case_name] = unmet_rsv_totals

    # calculate reg and rsv revenue sums
    cem_reg_revenue_sum = cem_raw['OperatingRegulationRevenue'].sum()
    cem_rsv_revenue_sum = cem_raw['OperatingReserveRevenue'].sum()

    pf_reg_revenue_sum = pf_raw['OperatingRegulationRevenue'].sum()
    pf_rsv_revenue_sum = pf_raw['OperatingReserveRevenue'].sum()

    dlac_reg_revenue_sum = dlac_raw['OperatingRegulationRevenue'].sum()
    dlac_rsv_revenue_sum = dlac_raw['OperatingReserveRevenue'].sum()

    dlac_imperfect_reg_revenue_sum = dlac_imperfect_raw['OperatingRegulationRevenue'].sum()
    dlac_imperfect_rsv_revenue_sum = dlac_imperfect_raw['OperatingReserveRevenue'].sum()

    slac_reg_revenue_sum = slac_raw['OperatingRegulationRevenue'].sum()
    slac_rsv_revenue_sum = slac_raw['OperatingReserveRevenue'].sum()



    # load unmet reserves cost information
    pf_ur_cost = pd.read_csv(pf_results_path + 'revenue_unmet_rsv_cost.csv', header=None)
    dlac_ur_cost = pd.read_csv(dlac_results_path + 'revenue_unmet_rsv_cost.csv', header=None)
    dlac_imperfect_ur_cost = pd.read_csv(dlac_imperfect_results_path + 'revenue_unmet_rsv_cost.csv', header=None)
    slac_ur_cost = pd.read_csv(slac_results_path + 'revenue_unmet_rsv_cost.csv', header=None)

    pf_nse_cost = pd.read_csv(pf_results_path + 'revenue_nse_cost.csv', header=None)
    dlac_nse_cost = pd.read_csv(dlac_results_path + 'revenue_nse_cost.csv', header=None)
    dlac_imperfect_nse_cost = pd.read_csv(dlac_imperfect_results_path + 'revenue_nse_cost.csv', header=None)
    slac_nse_cost = pd.read_csv(slac_results_path + 'revenue_nse_cost.csv', header=None)

    pf_sum_nse_cost = pf_nse_cost.sum(axis=1)[0]
    dlac_sum_nse_cost = dlac_nse_cost.sum(axis=1)[0]
    dlac_imperfect_sum_nse_cost = dlac_imperfect_nse_cost.sum(axis=1)[0]
    slac_sum_nse_cost = slac_nse_cost.sum(axis=1)[0]

    pf_sum_ur_cost = pf_ur_cost.sum(axis=1)[0]
    dlac_sum_ur_cost = dlac_ur_cost.sum(axis=1)[0]
    dlac_imperfect_sum_ur_cost = dlac_imperfect_ur_cost.sum(axis=1)[0]
    slac_sum_ur_cost = slac_ur_cost.sum(axis=1)[0]

    total_nse_cost_dic[case_name] = {
        'CEM': f"{cem_total_nse_cost:.4e}",
        'PF': f"{pf_sum_nse_cost:.4e}",
        'DLAC': f"{dlac_sum_nse_cost:.4e}",
        'DLAC Imperfect': f"{dlac_imperfect_sum_nse_cost:.4e}",
        'SLAC': f"{slac_sum_nse_cost:.4e}"
    }

    total_unmet_rsv_cost_dic[case_name] = {
        'CEM': f"{cem_sum_ur_cost:.4e}",
        'PF': f"{pf_sum_ur_cost:.4e}",
        'DLAC': f"{dlac_sum_ur_cost:.4e}",
        'DLAC Imperfect': f"{dlac_imperfect_sum_ur_cost:.4e}",
        'SLAC': f"{slac_sum_ur_cost:.2e}"
    }

    cem_prices = cem_raw_prices['1']
    pf_prices = pf_raw_prices[0]
    dlac_prices = dlac_raw_prices[0]
    dlac_imperfect_prices = dlac_imperfect_raw_prices[0]
    slac_prices = slac_raw_prices[0]

    cem_costs = cem_raw[cost_params]
    pf_costs = pf_raw[cost_params]
    dlac_costs = dlac_raw[cost_params]
    dlac_imperfect_costs = dlac_imperfect_raw[cost_params]
    slac_costs = slac_raw[cost_params]

    cem_economics = cem_raw[cost_params + revenue_params]
    pf_economics = pf_raw[cost_params + revenue_params]
    dlac_economics = dlac_raw[cost_params + revenue_params]
    dlac_imperfect_economics = dlac_imperfect_raw[cost_params + revenue_params]
    slac_economics = slac_raw[cost_params + revenue_params]

    cem_economics_copy = cem_economics.copy()
    pf_economics_copy = pf_economics.copy()
    dlac_economics_copy = dlac_economics.copy()
    dlac_imperfect_economics_copy = dlac_imperfect_economics.copy()
    slac_economics_copy = slac_economics.copy()

    cem_economics_copy[cost_params] = cem_economics_copy[cost_params] * -1
    pf_economics_copy[cost_params] = pf_economics_copy[cost_params] * -1
    dlac_economics_copy[cost_params] = dlac_economics_copy[cost_params] * -1
    dlac_imperfect_economics_copy[cost_params] = dlac_imperfect_economics_copy[cost_params] * -1
    slac_economics_copy[cost_params] = slac_economics_copy[cost_params] * -1

    cem_profit = cem_economics_copy.sum(axis=1)
    pf_profit = pf_economics_copy.sum(axis=1)
    dlac_profit = dlac_economics_copy.sum(axis=1)
    dlac_imperfect_profit = dlac_imperfect_economics_copy.sum(axis=1)
    slac_profit = slac_economics_copy.sum(axis=1)

    cem_economics_scaled = cem_economics_copy / ModelScalingFactor**2
    pf_economics_scaled = pf_economics_copy / ModelScalingFactor**2
    dlac_economics_scaled = dlac_economics_copy / ModelScalingFactor**2
    dlac_imperfect_economics_scaled = dlac_imperfect_economics_copy / ModelScalingFactor**2
    slac_economics_scaled = slac_economics_copy / ModelScalingFactor**2

    cem_scaled_profit = cem_economics_scaled.sum(axis=1)
    pf_scaled_profit = pf_economics_scaled.sum(axis=1)
    dlac_scaled_profit = dlac_economics_scaled.sum(axis=1)
    dlac_imperfect_scaled_profit = dlac_imperfect_economics_scaled.sum(axis=1)
    slac_scaled_profit = slac_economics_scaled.sum(axis=1)

    ###================================================================================================
    ### Total Investment Costs
    ###================================================================================================
    cem_mw_investment_costs = cem_raw['Inv_cost_MW'].sum()
    cem_mwhour_investment_costs = cem_raw['Inv_cost_MWh'].sum()

    pf_mw_investment_costs = pf_raw['Inv_cost_MW'].sum()
    pf_mwhour_investment_costs = pf_raw['Inv_cost_MWh'].sum()

    dlac_mw_investment_costs = dlac_raw['Inv_cost_MW'].sum()
    dlac_mwhour_investment_costs = dlac_raw['Inv_cost_MWh'].sum()

    dlac_imperfect_mw_investment_costs = dlac_imperfect_raw['Inv_cost_MW'].sum()
    dlac_imperfect_mwhour_investment_costs = dlac_imperfect_raw['Inv_cost_MWh'].sum()

    slac_mw_investment_costs = slac_raw['Inv_cost_MW'].sum()
    slac_mwhour_investment_costs = slac_raw['Inv_cost_MWh'].sum()

    cem_total_investment_costs = cem_mw_investment_costs + cem_mwhour_investment_costs
    pf_total_investment_costs = pf_mw_investment_costs + pf_mwhour_investment_costs
    dlac_total_investment_costs = dlac_mw_investment_costs + dlac_mwhour_investment_costs
    dlac_imperfect_total_investment_costs = dlac_imperfect_mw_investment_costs + dlac_imperfect_mwhour_investment_costs
    slac_total_investment_costs = slac_mw_investment_costs + slac_mwhour_investment_costs

    cem_total_investment_costs_sci = f"{cem_total_investment_costs:.4e}"
    pf_total_investment_costs_sci = f"{pf_total_investment_costs:.4e}"
    dlac_total_investment_costs_sci = f"{dlac_total_investment_costs:.4e}"
    dlac_imperfect_total_investment_costs_sci = f"{dlac_imperfect_total_investment_costs:.4e}"
    slac_total_investment_costs_sci = f"{slac_total_investment_costs:.4e}"

    total_investment_costs = {
        'CEM': cem_total_investment_costs_sci,
        'PF': pf_total_investment_costs_sci,
        'DLAC': dlac_total_investment_costs_sci,
        'DLAC Imperfect': dlac_imperfect_total_investment_costs_sci,
        'SLAC': slac_total_investment_costs_sci
    }
    total_investment_costs_dic[case_name] = total_investment_costs


    ###================================================================================================
    ### Unit Profits
    ###================================================================================================

    cem_unit_profit_mw = cem_profit / gen_capacity_mw
    pf_unit_profit_mw = pf_profit / gen_capacity_mw
    dlac_unit_profit_mw = dlac_profit / gen_capacity_mw
    dlac_imperfect_unit_profit_mw = dlac_imperfect_profit / gen_capacity_mw
    slac_unit_profit_mw = slac_profit / gen_capacity_mw

    cem_unit_profit_mw_sci = cem_unit_profit_mw.apply(lambda x: f"{x:.4e}")
    pf_unit_profit_mw_sci = pf_unit_profit_mw.apply(lambda x: f"{x:.4e}")
    dlac_unit_profit_mw_sci = dlac_unit_profit_mw.apply(lambda x: f"{x:.4e}")
    dlac_imperfect_unit_profit_mw_sci = dlac_imperfect_unit_profit_mw.apply(lambda x: f"{x:.4e}")
    slac_unit_profit_mw_sci = slac_unit_profit_mw.apply(lambda x: f"{x:.4e}")

    cem_unit_profit_gw = cem_profit / gen_capacity_gw
    pf_unit_profit_gw = pf_profit / gen_capacity_gw
    dlac_unit_profit_gw = dlac_profit / gen_capacity_gw
    dlac_imperfect_unit_profit_gw = dlac_imperfect_profit / gen_capacity_gw
    slac_unit_profit_gw = slac_profit / gen_capacity_gw

    unit_profit_df = pd.DataFrame({
        'Resource': gen_print_names,
        'CEM': cem_unit_profit_mw_sci,
        'PF': pf_unit_profit_mw_sci,
        'DLAC': dlac_unit_profit_mw_sci,
        'DLAC Imperfect': dlac_imperfect_unit_profit_mw_sci,
        'SLAC': slac_unit_profit_mw_sci
    })

    # sorted_unit_profits = sorted(unit_profits.items(), key=lambda item: item[1], reverse=True)

    unit_profit_dic[case_name] = unit_profit_df

    ###================================================================================================
    ### Calculate consumer payments
    ###================================================================================================

    cem_consumer_payments = cem_prices * load_data
    pf_consumer_payments = pf_prices * load_data
    dlac_consumer_payments = dlac_prices * load_data
    dlac_imperfect_consumer_payments = dlac_imperfect_prices * load_data 
    slac_consumer_payments = slac_prices * load_data

    sum_cem_cp = cem_consumer_payments.sum() + cem_reg_revenue_sum + cem_rsv_revenue_sum
    sum_pf_cp = pf_consumer_payments.sum() + pf_reg_revenue_sum + pf_rsv_revenue_sum
    sum_dlac_cp = dlac_consumer_payments.sum() + dlac_reg_revenue_sum + dlac_rsv_revenue_sum
    sum_dlac_imperfect_cp = dlac_imperfect_consumer_payments.sum() + \
                                    dlac_imperfect_reg_revenue_sum + dlac_imperfect_rsv_revenue_sum
    sum_slac_cp = slac_consumer_payments.sum()  + slac_reg_revenue_sum + slac_rsv_revenue_sum

    # # Convert sums to millions of dollars
    # sum_cem_cp_mil = sum_cem_cp / 1e6
    # sum_dlac_cp_mil = sum_dlac_cp / 1e6
    # sum_dlac_imperfect_cp_mil = sum_dlac_imperfect_cp / 1e6
    # sum_slac_cp_mil = sum_slac_cp / 1e6

    sum_cem_cp_sci = f"{sum_cem_cp:.4e}"
    sum_pf_cp_sci = f"{sum_pf_cp:.4e}"
    sum_dlac_cp_sci = f"{sum_dlac_cp:.4e}"
    sum_dlac_imperfect_cp_sci = f"{sum_dlac_imperfect_cp:.4e}"
    sum_slac_cp_sci = f"{sum_slac_cp:.4e}"

    # Create a dictionary with the sums
    consumer_payments = {
        'CEM': sum_cem_cp_sci,
        'PF': sum_pf_cp_sci,
        'DLAC': sum_dlac_cp_sci,
        'DLAC Imperfect': sum_dlac_imperfect_cp_sci,
        'SLAC': sum_slac_cp_sci
    }

    # # Sort the dictionary by values in descending order
    # sorted_consumer_payments = sorted(consumer_payments.items(), key=lambda item: item[1], reverse=True)

    consumer_payments_dic[case_name] = consumer_payments

    ###================================================================================================
    ### Calculate Total System Costs
    ###================================================================================================

    # load unmet reserves cost information
    pf_ur_cost = pd.read_csv(pf_results_path + 'revenue_unmet_rsv_cost.csv', header=None)
    dlac_ur_cost = pd.read_csv(dlac_results_path + 'revenue_unmet_rsv_cost.csv', header=None)
    dlac_imperfect_ur_cost = pd.read_csv(dlac_imperfect_results_path + 'revenue_unmet_rsv_cost.csv', header=None)
    slac_ur_cost = pd.read_csv(slac_results_path + 'revenue_unmet_rsv_cost.csv', header=None)

    pf_sum_ur_cost = pf_ur_cost.sum(axis=1)[0]
    dlac_sum_ur_cost = dlac_ur_cost.sum(axis=1)[0]
    dlac_imperfect_sum_ur_cost = dlac_imperfect_ur_cost.sum(axis=1)[0]
    slac_sum_ur_cost = slac_ur_cost.sum(axis=1)[0]

    # nse_costs_dic[case_name] = {} # XXX why is this here?

    cem_total_system_costs = cem_costs.sum().sum() + cem_total_nse_cost
    pf_total_system_costs = pf_costs.sum().sum() + pf_sum_nse_cost
    dlac_total_system_costs = dlac_costs.sum().sum() + dlac_sum_nse_cost
    dlac_imperfect_total_system_costs = dlac_imperfect_costs.sum().sum() + dlac_imperfect_sum_nse_cost
    slac_total_system_costs = slac_costs.sum().sum() + slac_sum_nse_cost

    cem_total_system_costs_sci = f"{cem_total_system_costs:.4e}"
    pf_total_system_costs_sci = f"{pf_total_system_costs:.4e}"
    dlac_total_system_costs_sci = f"{dlac_total_system_costs:.4e}"
    dlac_imperfect_total_system_costs_sci = f"{dlac_imperfect_total_system_costs:.4e}"
    slac_total_system_costs_sci = f"{slac_total_system_costs:.4e}"

    total_inv_op_costs = {
        'CEM': f"{cem_costs.sum().sum():.4e}",
        'PF': f"{pf_costs.sum().sum():.4e}",
        'DLAC': f"{dlac_costs.sum().sum():.4e}",
        'DLAC Imperfect': f"{dlac_imperfect_costs.sum().sum():.4e}",
        'SLAC': f"{slac_costs.sum().sum():.4e}"
    }

    total_inv_op_costs_dic[case_name] = total_inv_op_costs

    # Create a dictionary with the total costs
    total_system_costs = {
        'CEM': cem_total_system_costs_sci,
        'PF': pf_total_system_costs_sci,
        'DLAC': dlac_total_system_costs_sci,
        'DLAC Imperfect': dlac_imperfect_total_system_costs_sci,
        'SLAC': slac_total_system_costs_sci
    }

    # # Sort the dictionary by values in descending order
    # sorted_total_system_costs = sorted(total_system_costs.items(), key=lambda item: item[1], reverse=True)

    # # Print the sorted total costs
    # for case, cost in sorted_total_system_costs:
    #     print("case_name: " f"{case}: {cost:.4e}")

    total_system_costs_dic[case_name] = total_system_costs

    ###================================================================================================
    ### Calculate Market Surplus/// Costs
    ###================================================================================================
    cem_objective_total = cem_total_system_costs + cem_sum_ur_cost
    pf_objective_total = pf_total_system_costs + pf_sum_ur_cost
    dlac_objective_total = dlac_total_system_costs + dlac_sum_ur_cost
    dlac_imperfect_objective_total = dlac_imperfect_total_system_costs + dlac_imperfect_sum_ur_cost
    slac_objective_total = slac_total_system_costs + slac_sum_ur_cost

    cem_objective_total_sci = f"{cem_objective_total:.4e}"
    pf_objective_total_sci = f"{pf_objective_total:.4e}"
    dlac_objective_total_sci = f"{dlac_objective_total:.4e}"
    dlac_imperfect_objective_total_sci = f"{dlac_imperfect_objective_total:.4e}"
    slac_objective_total_sci = f"{slac_objective_total:.4e}"

    # Create a dictionary with the total costs
    total_objective = {
        'CEM': cem_objective_total_sci,
        'PF': pf_objective_total_sci,
        'DLAC': dlac_objective_total_sci,
        'DLAC Imperfect': dlac_imperfect_objective_total_sci,
        'SLAC': slac_objective_total_sci
    }

    total_objective_dic[case_name] = total_objective

    ###================================================================================================
    ### Calculate Operating Costs
    ###================================================================================================
    operating_costs = ['Fixed_OM_cost_MW', 'Var_OM_cost_out', 'Fuel_cost', 'Var_OM_cost_in', 'StartCost', 'Charge_cost']

    cem_operating_costs = cem_costs[operating_costs].sum().sum()
    pf_operating_costs = pf_costs[operating_costs].sum().sum()
    dlac_operating_costs = dlac_costs[operating_costs].sum().sum()
    dlac_imperfect_operating_costs = dlac_imperfect_costs[operating_costs].sum().sum()
    slac_operating_costs = slac_costs[operating_costs].sum().sum()

    cem_operating_costs_sci = f"{cem_operating_costs:.4e}"
    pf_operating_costs_sci = f"{pf_operating_costs:.4e}"
    dlac_operating_costs_sci = f"{dlac_operating_costs:.4e}"
    dlac_imperfect_operating_costs_sci = f"{dlac_imperfect_operating_costs:.4e}"
    slac_operating_costs_sci = f"{slac_operating_costs:.4e}"

    # Create a dictionary with the operating costs
    total_operating_costs = {
        'CEM': cem_operating_costs_sci,
        'PF': pf_operating_costs_sci,
        'DLAC': dlac_operating_costs_sci,
        'DLAC Imperfect': dlac_imperfect_operating_costs_sci,
        'SLAC': slac_operating_costs_sci
    }

    total_operating_costs_dic[case_name] = total_operating_costs

    ###================================================================================================
    ### Calculate Fixed OM Costs
    ###================================================================================================

    cem_fixed_om_costs = cem_costs['Fixed_OM_cost_MW'].sum().sum()
    pf_fixed_om_costs = pf_costs['Fixed_OM_cost_MW'].sum().sum()
    dlac_fixed_om_costs = dlac_costs['Fixed_OM_cost_MW'].sum().sum()
    dlac_imperfect_fixed_om_costs = dlac_imperfect_costs['Fixed_OM_cost_MW'].sum().sum()
    slac_fixed_om_costs = slac_costs['Fixed_OM_cost_MW'].sum().sum()

    cem_fixed_om_costs_sci = f"{cem_fixed_om_costs:.4e}"
    pf_fixed_om_costs_sci = f"{pf_fixed_om_costs:.4e}"
    dlac_fixed_om_costs_sci = f"{dlac_fixed_om_costs:.4e}"
    dlac_imperfect_fixed_om_costs_sci = f"{dlac_imperfect_fixed_om_costs:.4e}"
    slac_fixed_om_costs_sci = f"{slac_fixed_om_costs:.4e}"

    # Create a dictionary with the fixed OM costs
    total_fixed_om_costs = {
        'CEM': cem_fixed_om_costs_sci,
        'PF': pf_fixed_om_costs_sci,
        'DLAC': dlac_fixed_om_costs_sci,
        'DLAC Imperfect': dlac_imperfect_fixed_om_costs_sci,
        'SLAC': slac_fixed_om_costs_sci
    }

    total_fixed_om_costs_dic[case_name] = total_fixed_om_costs

    ###================================================================================================
    ### Calculate Variable OM Costs
    ###================================================================================================

    cem_var_om_costs = cem_costs['Var_OM_cost_out'].sum().sum()
    pf_var_om_costs = pf_costs['Var_OM_cost_out'].sum().sum()
    dlac_var_om_costs = dlac_costs['Var_OM_cost_out'].sum().sum()
    dlac_imperfect_var_om_costs = dlac_imperfect_costs['Var_OM_cost_out'].sum().sum()
    slac_var_om_costs = slac_costs['Var_OM_cost_out'].sum().sum()

    cem_var_om_costs_sci = f"{cem_var_om_costs:.4e}"
    pf_var_om_costs_sci = f"{pf_var_om_costs:.4e}"
    dlac_var_om_costs_sci = f"{dlac_var_om_costs:.4e}"
    dlac_imperfect_var_om_costs_sci = f"{dlac_imperfect_var_om_costs:.4e}"
    slac_var_om_costs_sci = f"{slac_var_om_costs:.4e}"

    # Create a dictionary with the variable OM costs
    total_var_om_costs = {
        'CEM': cem_var_om_costs_sci,
        'PF': pf_var_om_costs_sci,
        'DLAC': dlac_var_om_costs_sci,
        'DLAC Imperfect': dlac_imperfect_var_om_costs_sci,
        'SLAC': slac_var_om_costs_sci
    }

    total_var_om_out_costs_dic[case_name] = total_var_om_costs

    ###================================================================================================
    ### Calculate Fuel Costs
    ###================================================================================================

    cem_fuel_costs = cem_costs['Fuel_cost'].sum().sum()
    pf_fuel_costs = pf_costs['Fuel_cost'].sum().sum()
    dlac_fuel_costs = dlac_costs['Fuel_cost'].sum().sum()
    dlac_imperfect_fuel_costs = dlac_imperfect_costs['Fuel_cost'].sum().sum()
    slac_fuel_costs = slac_costs['Fuel_cost'].sum().sum()

    cem_fuel_costs_sci = f"{cem_fuel_costs:.4e}"
    pf_fuel_costs_sci = f"{pf_fuel_costs:.4e}"
    dlac_fuel_costs_sci = f"{dlac_fuel_costs:.4e}"
    dlac_imperfect_fuel_costs_sci = f"{dlac_imperfect_fuel_costs:.4e}"
    slac_fuel_costs_sci = f"{slac_fuel_costs:.4e}"

    # Create a dictionary with the fuel costs
    total_fuel_costs = {
        'CEM': cem_fuel_costs_sci,
        'PF': pf_fuel_costs_sci,
        'DLAC': dlac_fuel_costs_sci,
        'DLAC Imperfect': dlac_imperfect_fuel_costs_sci,
        'SLAC': slac_fuel_costs_sci
    }

    total_fuel_costs_dic[case_name] = total_fuel_costs

    ###================================================================================================
    ### Calculate Variable OM Costs
    ###================================================================================================

    cem_var_om_in_costs = cem_costs['Var_OM_cost_in'].sum().sum()
    pf_var_om_in_costs = pf_costs['Var_OM_cost_in'].sum().sum()
    dlac_var_om_in_costs = dlac_costs['Var_OM_cost_in'].sum().sum()
    dlac_imperfect_var_om_in_costs = dlac_imperfect_costs['Var_OM_cost_in'].sum().sum()
    slac_var_om_in_costs = slac_costs['Var_OM_cost_in'].sum().sum()

    cem_var_om_in_costs_sci = f"{cem_var_om_in_costs:.4e}"
    pf_var_om_in_costs_sci = f"{pf_var_om_in_costs:.4e}"
    dlac_var_om_in_costs_sci = f"{dlac_var_om_in_costs:.4e}"
    dlac_imperfect_var_om_in_costs_sci = f"{dlac_imperfect_var_om_in_costs:.4e}"
    slac_var_om_in_costs_sci = f"{slac_var_om_in_costs:.4e}"

    # Create a dictionary with the variable OM costs
    total_var_om_in_costs = {
        'CEM': cem_var_om_in_costs_sci,
        'PF': pf_var_om_in_costs_sci,
        'DLAC': dlac_var_om_in_costs_sci,
        'DLAC Imperfect': dlac_imperfect_var_om_in_costs_sci,
        'SLAC': slac_var_om_in_costs_sci
    }

    total_var_om_in_costs_dic[case_name] = total_var_om_in_costs

    ###================================================================================================
    ### Calculate Start Costs
    ###================================================================================================

    cem_start_costs = cem_costs['StartCost'].sum().sum()
    pf_start_costs = pf_costs['StartCost'].sum().sum()
    dlac_start_costs = dlac_costs['StartCost'].sum().sum()
    dlac_imperfect_start_costs = dlac_imperfect_costs['StartCost'].sum().sum()
    slac_start_costs = slac_costs['StartCost'].sum().sum()

    cem_start_costs_sci = f"{cem_start_costs:.4e}"
    pf_start_costs_sci = f"{pf_start_costs:.4e}"
    dlac_start_costs_sci = f"{dlac_start_costs:.4e}"
    dlac_imperfect_start_costs_sci = f"{dlac_imperfect_start_costs:.4e}"
    slac_start_costs_sci = f"{slac_start_costs:.4e}"

    # Create a dictionary with the start costs
    total_start_costs = {
        'CEM': cem_start_costs_sci,
        'PF': pf_start_costs_sci,
        'DLAC': dlac_start_costs_sci,
        'DLAC Imperfect': dlac_imperfect_start_costs_sci,
        'SLAC': slac_start_costs_sci
    }

    total_start_costs_dic[case_name] = total_start_costs

    ###================================================================================================
    ### Calculate Charge Costs
    ###================================================================================================

    cem_charge_costs = cem_costs['Charge_cost'].sum().sum()
    pf_charge_costs = pf_costs['Charge_cost'].sum().sum()
    dlac_charge_costs = dlac_costs['Charge_cost'].sum().sum()
    dlac_imperfect_charge_costs = dlac_imperfect_costs['Charge_cost'].sum().sum()
    slac_charge_costs = slac_costs['Charge_cost'].sum().sum()

    cem_charge_costs_sci = f"{cem_charge_costs:.4e}"
    pf_charge_costs_sci = f"{pf_charge_costs:.4e}"
    dlac_charge_costs_sci = f"{dlac_charge_costs:.4e}"
    dlac_imperfect_charge_costs_sci = f"{dlac_imperfect_charge_costs:.4e}"
    slac_charge_costs_sci = f"{slac_charge_costs:.4e}"

    # Create a dictionary with the charge costs
    total_charge_costs = {
        'CEM': cem_charge_costs_sci,
        'PF': pf_charge_costs_sci,
        'DLAC': dlac_charge_costs_sci,
        'DLAC Imperfect': dlac_imperfect_charge_costs_sci,
        'SLAC': slac_charge_costs_sci
    }

    total_charge_costs_dic[case_name] = total_charge_costs

    ###================================================================================================
    ### Calculate Profit Margin Rate
    ###================================================================================================

    # dfGen.index = gen_print_names
    char_str = ['Inv_Cost_per_MWyr', 'Inv_Cost_per_MWhyr', 'Fixed_OM_Cost_per_MWyr']
    dfGen_annual_costs = dfGen[char_str]

    # if the Inv_Cost_per_MWhyr in each row is > 0, then multiply the Inv_Cost_per_MWhyr by the Max_Duration
    inv_cost_per_mwhyr2mwyr = dfGen.apply(
        lambda row: row['Inv_Cost_per_MWhyr'] * row['Existing_Cap_MWh'] / row['Existing_Cap_MW'] \
            if row['Inv_Cost_per_MWhyr'] > 0 else row['Inv_Cost_per_MWhyr'],
        axis=1
    )

    # Replace 'Inv_Cost_per_MWhyr' in dfGen_annual_costs with inv_cost_permwhyr2mwyr and title it 'Inv_Cost_per_Energy_by_hour'
    dfGen_annual_costs['Inv_Cost_per_Energy_by_hour'] = inv_cost_per_mwhyr2mwyr
    dfGen_annual_costs.drop(columns=['Inv_Cost_per_MWhyr'], inplace=True)

    dfGen_annual_costs_sum = dfGen_annual_costs.sum(axis=1) # $/MWyr

    # \alpha_{i,\Pi} = \frac{P_{i,\Pi}}{C_{i,\Pi}}
    cem_annual_profit2costs = cem_unit_profit_mw / dfGen_annual_costs_sum
    pf_annual_profit2costs = pf_unit_profit_mw / dfGen_annual_costs_sum
    dlac_annual_profit2costs = dlac_unit_profit_mw / dfGen_annual_costs_sum
    dlac_imperfect_annual_profit2costs = dlac_imperfect_unit_profit_mw / dfGen_annual_costs_sum
    slac_annual_profit2costs = slac_unit_profit_mw / dfGen_annual_costs_sum

    cem_ap2c_perc = cem_annual_profit2costs * 100
    pf_ap2c_perc = pf_annual_profit2costs * 100
    dlac_ap2c_perc = dlac_annual_profit2costs * 100
    dlac_imperfect_ap2c_perc = dlac_imperfect_annual_profit2costs * 100
    slac_ap2c_perc = slac_annual_profit2costs * 100

    cem_ap2c_perc_sci = cem_ap2c_perc.apply(lambda x: f"{x:.2f}%")
    pf_ap2c_perc_sci = pf_ap2c_perc.apply(lambda x: f"{x:.2f}%")
    dlac_ap2c_perc_sci = dlac_ap2c_perc.apply(lambda x: f"{x:.2f}%")
    dlac_imperfect_perc_ap2c_sci = dlac_imperfect_ap2c_perc.apply(lambda x: f"{x:.2f}%")
    slac_ap2c_perc_sci = slac_ap2c_perc.apply(lambda x: f"{x:.2f}%")


    percentage_profit_df = pd.DataFrame({
        'Resource': gen_print_names,
        'CEM': cem_ap2c_perc_sci,
        'PF': pf_ap2c_perc_sci,
        'DLAC': dlac_ap2c_perc_sci,
        'DLAC Imperfect': dlac_imperfect_perc_ap2c_sci,
        'SLAC': slac_ap2c_perc_sci
    })

    percentage_profits_dic[case_name] = percentage_profit_df
    ### DONE


Case Name: Thermal_Base

['Coal-new' 'HTGR' 'Microreactor' 'NG 2-on-1 Combined Cycle (F-Frame)'
 'NG 2-on-1 Combined Cycle (H-Frame) 95% CCS'
 'NG Combustion Turbine (F-Frame)' 'PBR-HTGR' 'RICE' 'iPWR_Pack'
 'Land-Based Wind - Class 1 - Technology 1' 'Utility PV - Class 1'
 'Utility-Scale Battery Storage - 4Hr']


C:\Users\ks885\AppData\Local\Temp\ipykernel_9500\1224991645.py:650: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfGen_annual_costs['Inv_Cost_per_Energy_by_hour'] = inv_cost_per_mwhyr2mwyr
C:\Users\ks885\AppData\Local\Temp\ipykernel_9500\1224991645.py:651: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfGen_annual_costs.drop(columns=['Inv_Cost_per_MWhyr'], inplace=True)


In [177]:
def percent_change_df(df):
    norm = df.copy()
    norm['CEM'] = df['CEM'].astype(float) / df['PF'].astype(float)
    norm['PF'] = df['PF'].astype(float) / df['PF'].astype(float)
    norm['DLAC'] = df['DLAC'].astype(float) / df['PF'].astype(float)
    norm['DLAC Imperfect'] = df['DLAC Imperfect'].astype(float) / df['PF'].astype(float)
    norm['SLAC'] = df['SLAC'].astype(float) / df['PF'].astype(float)

    change_df = norm.copy()
    change_df['CEM'] = norm['CEM'] - 1
    change_df['PF'] = norm['PF'] - 1
    change_df['DLAC'] = norm['DLAC'] - 1
    change_df['DLAC Imperfect'] = norm['DLAC Imperfect'] - 1
    change_df['SLAC'] = norm['SLAC'] - 1

    percent_change_df = change_df.copy()
    percent_change_df['CEM'] = change_df['CEM'] * 100
    percent_change_df['PF'] = change_df['PF'] * 100
    percent_change_df['DLAC'] = change_df['DLAC'] * 100
    percent_change_df['DLAC Imperfect'] = change_df['DLAC Imperfect'] * 100
    percent_change_df['SLAC'] = change_df['SLAC'] * 100

    # remove decimal point
    percent_change_df[['CEM', 'PF', 'DLAC', 'DLAC Imperfect', 'SLAC']] = \
                        percent_change_df[['CEM', 'PF', 'DLAC', 'DLAC Imperfect', 'SLAC']].round(2)
    percent_change_df[['CEM', 'PF', 'DLAC', 'DLAC Imperfect', 'SLAC']] = \
        percent_change_df[['CEM', 'PF', 'DLAC', 'DLAC Imperfect', 'SLAC']].astype(str) + '%'

    return percent_change_df


In [178]:
# create dataframe from total_nse_dic with rows for each case_name in case_names
total_nse_df = pd.DataFrame.from_dict(total_nse_dic, orient='index').reset_index()
total_nse_df.columns = ['Case Name', 'CEM', 'PF', 'DLAC', 'DLAC Imperfect', 'SLAC']
total_nse_df.to_csv(tables_path + 'total_nse.csv', index=False)
total_nse_df

,Case Name,CEM,PF,DLAC,DLAC Imperfect,SLAC
0,Thermal_Base,1.1753e+04,1.1753e+04,1.1753e+04,1.7483e+04,1.6990e+04


In [179]:
nse_change_df = percent_change_df(total_nse_df)
nse_change_df

,Case Name,CEM,PF,DLAC,DLAC Imperfect,SLAC
0,Thermal_Base,0.0%,0.0%,0.0%,48.75%,44.56%


In [180]:
total_unmet_rsv_df = pd.DataFrame.from_dict(total_unmet_rsv_dic, orient='index').reset_index()
total_unmet_rsv_df.columns = ['Case Name', 'CEM', 'PF', 'DLAC', 'DLAC Imperfect', 'SLAC']
total_unmet_rsv_df.to_csv(tables_path + 'total_unmet_rsv.csv', index=False)
total_unmet_rsv_df

,Case Name,CEM,PF,DLAC,DLAC Imperfect,SLAC
0,Thermal_Base,3.7025e+04,3.7025e+04,3.7025e+04,4.0262e+04,4.0984e+04


In [181]:
# Create a dataframe from consumer_payments_dic with rows for each case_name in case_names
consumer_payments_df = pd.DataFrame.from_dict(consumer_payments_dic, orient='index').reset_index()
consumer_payments_df.columns = ['Case Name', 'CEM', 'PF', 'DLAC', 'DLAC Imperfect', 'SLAC']
consumer_payments_df.to_csv(tables_path + 'consumer_payments.csv', index=False)
consumer_payments_df

,Case Name,CEM,PF,DLAC,DLAC Imperfect,SLAC
0,Thermal_Base,1.8035e+10,1.8355e+10,1.7381e+10,1.8471e+10,1.6301e+10


In [182]:
consumer_payments_change_df = percent_change_df(consumer_payments_df)
consumer_payments_change_df

,Case Name,CEM,PF,DLAC,DLAC Imperfect,SLAC
0,Thermal_Base,-1.74%,0.0%,-5.31%,0.63%,-11.19%


In [183]:
total_inv_costs_df = pd.DataFrame.from_dict(total_investment_costs_dic, orient='index').reset_index()
total_inv_costs_df.columns = ['Case Name', 'CEM', 'PF', 'DLAC', 'DLAC Imperfect', 'SLAC']
total_inv_costs_df.to_csv(tables_path + 'total_inv_costs.csv', index=False)
total_inv_costs_df

,Case Name,CEM,PF,DLAC,DLAC Imperfect,SLAC
0,Thermal_Base,7.1435e+09,7.1435e+09,7.1435e+09,7.1435e+09,7.1435e+09


In [184]:
total_inv_cost_change = percent_change_df(total_inv_costs_df)
total_inv_cost_change

,Case Name,CEM,PF,DLAC,DLAC Imperfect,SLAC
0,Thermal_Base,0.0%,0.0%,0.0%,0.0%,0.0%


In [185]:
total_inv_op_costs_df = pd.DataFrame.from_dict(total_inv_op_costs_dic, orient='index').reset_index()
total_inv_op_costs_df.columns = ['Case Name', 'CEM', 'PF', 'DLAC', 'DLAC Imperfect', 'SLAC']
total_inv_op_costs_df.to_csv(tables_path + 'total_inv_op_costs.csv', index=False)
total_inv_op_costs_df

,Case Name,CEM,PF,DLAC,DLAC Imperfect,SLAC
0,Thermal_Base,1.7977e+10,1.8192e+10,1.7776e+10,1.1737e+10,1.1718e+10


In [186]:
# Create a dataframe from total_system_costs_dic with rows for each case_name in case_names
total_operating_costs_df = pd.DataFrame.from_dict(total_operating_costs_dic, orient='index').reset_index()
total_operating_costs_df.columns = ['Case Name', 'CEM', 'PF', 'DLAC', 'DLAC Imperfect', 'SLAC']
total_operating_costs_df.to_csv(tables_path + 'total_operational_costs.csv', index=False)
total_operating_costs_df

,Case Name,CEM,PF,DLAC,DLAC Imperfect,SLAC
0,Thermal_Base,1.0833e+10,1.1049e+10,1.0632e+10,4.5931e+09,4.5744e+09


In [187]:
operating_cost_change_df = percent_change_df(total_operating_costs_df)
operating_cost_change_df

,Case Name,CEM,PF,DLAC,DLAC Imperfect,SLAC
0,Thermal_Base,-1.95%,0.0%,-3.77%,-58.43%,-58.6%


In [188]:
total_fixed_om_costs_df = pd.DataFrame.from_dict(total_fixed_om_costs_dic, orient='index').reset_index()
total_fixed_om_costs_df.columns = ['Case Name', 'CEM', 'PF', 'DLAC', 'DLAC Imperfect', 'SLAC']
total_fixed_om_costs_df.to_csv(tables_path + 'total_fixed_om_costs.csv', index=False)
total_fixed_om_costs_df

,Case Name,CEM,PF,DLAC,DLAC Imperfect,SLAC
0,Thermal_Base,2.8734e+09,2.8734e+09,2.8734e+09,2.8734e+09,2.8734e+09


In [189]:
total_fixed_om_costs_change_df = percent_change_df(total_fixed_om_costs_df)
total_fixed_om_costs_change_df

,Case Name,CEM,PF,DLAC,DLAC Imperfect,SLAC
0,Thermal_Base,0.0%,0.0%,0.0%,0.0%,0.0%


In [190]:
total_var_om_out_costs_df = pd.DataFrame.from_dict(total_var_om_out_costs_dic, orient='index').reset_index()
total_var_om_out_costs_df.columns = ['Case Name', 'CEM', 'PF', 'DLAC', 'DLAC Imperfect', 'SLAC']
total_var_om_out_costs_df.to_csv(tables_path + 'total_var_om_out_costs.csv', index=False)
total_var_om_out_costs_df

,Case Name,CEM,PF,DLAC,DLAC Imperfect,SLAC
0,Thermal_Base,3.7762e+08,3.7714e+08,3.7733e+08,3.7053e+08,3.7763e+08


In [191]:
total_var_om_out_costs_change_df = percent_change_df(total_var_om_out_costs_df)
total_var_om_out_costs_change_df

,Case Name,CEM,PF,DLAC,DLAC Imperfect,SLAC
0,Thermal_Base,0.13%,0.0%,0.05%,-1.75%,0.13%


In [192]:
total_fuel_costs_df = pd.DataFrame.from_dict(total_fuel_costs_dic, orient='index').reset_index()
total_fuel_costs_df.columns = ['Case Name', 'CEM', 'PF', 'DLAC', 'DLAC Imperfect', 'SLAC']
total_fuel_costs_df.to_csv(tables_path + 'total_fuel_costs.csv', index=False)
total_fuel_costs_df

,Case Name,CEM,PF,DLAC,DLAC Imperfect,SLAC
0,Thermal_Base,7.3673e+09,7.3799e+09,7.3813e+09,1.1335e+09,1.1577e+09


In [193]:
total_fuel_costs_change_df = percent_change_df(total_fuel_costs_df)
total_fuel_costs_change_df

,Case Name,CEM,PF,DLAC,DLAC Imperfect,SLAC
0,Thermal_Base,-0.17%,0.0%,0.02%,-84.64%,-84.31%


In [194]:
total_var_om_in_costs_df = pd.DataFrame.from_dict(total_var_om_in_costs_dic, orient='index').reset_index()
total_var_om_in_costs_df.columns = ['Case Name', 'CEM', 'PF', 'DLAC', 'DLAC Imperfect', 'SLAC']
total_var_om_in_costs_df.to_csv(tables_path + 'total_var_om_in_costs.csv', index=False)
total_var_om_in_costs_df

,Case Name,CEM,PF,DLAC,DLAC Imperfect,SLAC
0,Thermal_Base,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00


In [195]:
total_var_om_in_costs_change_df = percent_change_df(total_var_om_in_costs_df)
total_var_om_in_costs_change_df

,Case Name,CEM,PF,DLAC,DLAC Imperfect,SLAC
0,Thermal_Base,nan%,nan%,nan%,nan%,nan%


In [196]:
total_inv_op_costs_change_df = percent_change_df(total_inv_op_costs_df)
total_inv_op_costs_change_df.to_csv(tables_path + 'total_inv_op_costs_change.csv', index=False)
total_inv_op_costs_change_df

,Case Name,CEM,PF,DLAC,DLAC Imperfect,SLAC
0,Thermal_Base,-1.18%,0.0%,-2.29%,-35.48%,-35.59%


In [197]:
total_unmet_rsv_cost_df = pd.DataFrame.from_dict(total_unmet_rsv_cost_dic, orient='index').reset_index()
total_unmet_rsv_cost_df.columns = ['Case Name', 'CEM', 'PF', 'DLAC', 'DLAC Imperfect', 'SLAC']
total_unmet_rsv_cost_df.to_csv(tables_path + 'total_unmet_rsv_costs.csv', index=False)
total_unmet_rsv_cost_df

,Case Name,CEM,PF,DLAC,DLAC Imperfect,SLAC
0,Thermal_Base,3.7025e+07,3.7025e+07,3.7025e+07,4.0262e+07,4.10e+07


In [198]:
total_change_unmet_rsv = percent_change_df(total_unmet_rsv_cost_df)
total_change_unmet_rsv

,Case Name,CEM,PF,DLAC,DLAC Imperfect,SLAC
0,Thermal_Base,0.0%,0.0%,0.0%,8.74%,10.74%


In [199]:
total_nse_cost_df = pd.DataFrame.from_dict(total_nse_cost_dic, orient='index').reset_index()
total_nse_cost_df.columns = ['Case Name', 'PF', 'CEM', 'DLAC', 'DLAC Imperfect', 'SLAC']
total_nse_cost_df.to_csv(tables_path + 'total_nse_costs.csv', index=False)
total_nse_cost_df

,Case Name,PF,CEM,DLAC,DLAC Imperfect,SLAC
0,Thermal_Base,5.8767e+07,5.8767e+07,5.8767e+07,8.7413e+07,8.4949e+07


In [200]:
nse_cost_change_df = percent_change_df(total_nse_cost_df)
nse_cost_change_df

,Case Name,PF,CEM,DLAC,DLAC Imperfect,SLAC
0,Thermal_Base,0.0%,0.0%,0.0%,48.75%,44.55%


In [201]:
# Create a dataframe from total_system_costs_dic with rows for each case_name in case_names
total_system_costs_df = pd.DataFrame.from_dict(total_system_costs_dic, orient='index').reset_index()
total_system_costs_df.columns = ['Case Name', 'CEM', 'PF', 'DLAC', 'DLAC Imperfect', 'SLAC']
total_system_costs_df.to_csv(tables_path + 'total_system_costs.csv', index=False)
total_system_costs_df

,Case Name,CEM,PF,DLAC,DLAC Imperfect,SLAC
0,Thermal_Base,1.8035e+10,1.8251e+10,1.7834e+10,1.1824e+10,1.1803e+10


In [202]:
total_system_costs_change_df = percent_change_df(total_system_costs_df)
total_system_costs_change_df

,Case Name,CEM,PF,DLAC,DLAC Imperfect,SLAC
0,Thermal_Base,-1.18%,0.0%,-2.28%,-35.21%,-35.33%


In [203]:
total_objective_df = pd.DataFrame.from_dict(total_objective_dic, orient='index').reset_index()
total_objective_df.columns = ['Case Name', 'CEM', 'PF', 'DLAC', 'DLAC Imperfect', 'SLAC']
total_objective_df.to_csv(tables_path + 'total_objective.csv', index=False)
total_objective_df

,Case Name,CEM,PF,DLAC,DLAC Imperfect,SLAC
0,Thermal_Base,1.8073e+10,1.8288e+10,1.7871e+10,1.1864e+10,1.1844e+10


In [204]:
total_objective_change_df = percent_change_df(total_objective_df)
total_objective_change_df

,Case Name,CEM,PF,DLAC,DLAC Imperfect,SLAC
0,Thermal_Base,-1.18%,0.0%,-2.28%,-35.13%,-35.24%


### These nse costs are calculated exogenously

In [205]:

# multiply the nse_totals by 5000 to get the total nse costs and compare against the total_nse_cost_df
nse_cost_from_totals = total_nse_df.copy()
for col in total_nse_cost_df.columns:
    if col != 'Case Name':
        nse_cost_from_totals[col] = nse_cost_from_totals[col].astype(float) * 5000

# Convert the values to scientific notation
for col in nse_cost_from_totals.columns:
    if col != 'Case Name':
        nse_cost_from_totals[col] = nse_cost_from_totals[col].apply(lambda x: f"{x:.4e}")

nse_cost_from_totals

,Case Name,CEM,PF,DLAC,DLAC Imperfect,SLAC
0,Thermal_Base,5.8765e+07,5.8765e+07,5.8765e+07,8.7415e+07,8.4950e+07


In [206]:
total_operating_costs_norm = total_operating_costs_df.copy()

total_operating_costs_norm['CEM'] = total_operating_costs_norm['CEM'].astype(float) / total_operating_costs_df['CEM'].astype(float)
total_operating_costs_norm['PF'] = total_operating_costs_norm['PF'].astype(float) / total_operating_costs_df['CEM'].astype(float)
total_operating_costs_norm['DLAC'] = total_operating_costs_norm['DLAC'].astype(float) / total_operating_costs_df['CEM'].astype(float)
total_operating_costs_norm['DLAC Imperfect'] = total_operating_costs_norm['DLAC Imperfect'].astype(float) / total_operating_costs_df['CEM'].astype(float)
total_operating_costs_norm['SLAC'] = total_operating_costs_norm['SLAC'].astype(float) / total_operating_costs_df['CEM'].astype(float)

total_operating_costs_norm.to_csv(tables_path + 'total_operating_costs_norm.csv', index=False)
total_operating_costs_norm

,Case Name,CEM,PF,DLAC,DLAC Imperfect,SLAC
0,Thermal_Base,1.0,1.019939,0.981446,0.423992,0.422265


In [207]:
# calculate total system cost normalized by the CEM total system cost
total_system_costs_normalized = total_system_costs_df.copy()
# for col in total_system_costs_normalized.columns[1:]:
#     total_system_costs_normalized[col] = pd.to_numeric(total_system_costs_df[col]) / pd.to_numeric(total_system_costs_df['CEM'])
#     #OR pd.to_numeric(total_system_costs_df[col], errors='coerce') / pd.to_numeric(total_system_costs_df['CEM'], errors='coerce')
#     total_system_costs_normalized[col] = total_system_costs_normalized[col].round(1)

total_system_costs_normalized['CEM'] = total_system_costs_df['CEM'].astype(float) / total_system_costs_df['CEM'].astype(float) - 1
total_system_costs_normalized['PF'] = total_system_costs_df['PF'].astype(float) / total_system_costs_df['CEM'].astype(float) - 1
total_system_costs_normalized['DLAC'] = total_system_costs_df['DLAC'].astype(float) / total_system_costs_df['CEM'].astype(float) - 1
total_system_costs_normalized['DLAC Imperfect'] = total_system_costs_df['DLAC Imperfect'].astype(float) / total_system_costs_df['CEM'].astype(float) - 1
total_system_costs_normalized['SLAC'] = total_system_costs_df['SLAC'].astype(float) / total_system_costs_df['CEM'].astype(float) - 1
total_system_costs_normalized

,Case Name,CEM,PF,DLAC,DLAC Imperfect,SLAC
0,Thermal_Base,0.0,0.011977,-0.011145,-0.344386,-0.34555


In [208]:
total_system_costs_norm_percentage = total_system_costs_normalized.copy()
total_system_costs_norm_percentage['CEM'] = total_system_costs_normalized['CEM'] * 100
total_system_costs_norm_percentage['PF'] = total_system_costs_normalized['PF'] * 100
total_system_costs_norm_percentage['DLAC'] = total_system_costs_normalized['DLAC'] * 100
total_system_costs_norm_percentage['DLAC Imperfect'] = total_system_costs_normalized['DLAC Imperfect'] * 100
total_system_costs_norm_percentage['SLAC'] = total_system_costs_normalized['SLAC'] * 100

# Remove the decimal point from the normalized total system costs
total_system_costs_norm_percentage [['CEM', 'DLAC', 'DLAC Imperfect', 'SLAC']] = \
    total_system_costs_norm_percentage [['CEM', 'DLAC', 'DLAC Imperfect', 'SLAC']].round(2)
# Add a '%' symbol at the end of the normalized total system costs
total_system_costs_norm_percentage [['CEM', 'DLAC', 'DLAC Imperfect', 'SLAC']] = \
    total_system_costs_norm_percentage [['CEM', 'DLAC', 'DLAC Imperfect', 'SLAC']].astype(str) + '%'

total_system_costs_norm_percentage .to_csv(tables_path + 'total_system_costs_norm_percentage .csv', index=False)
total_system_costs_norm_percentage 

,Case Name,CEM,PF,DLAC,DLAC Imperfect,SLAC
0,Thermal_Base,0.0%,1.197671,-1.11%,-34.44%,-34.56%


In [209]:
# subtract total costs_df from consumer payments df to get profits
# Convert the columns to float for calculation
consumer_payments_df[['CEM', 'PF', 'DLAC', 'DLAC Imperfect', 'SLAC']] = consumer_payments_df[['CEM', 'PF', 'DLAC', 'DLAC Imperfect', 'SLAC']].astype(float)
total_system_costs_df[['CEM', 'PF', 'DLAC', 'DLAC Imperfect', 'SLAC']] = total_system_costs_df[['CEM', 'PF', 'DLAC', 'DLAC Imperfect', 'SLAC']].astype(float)

# Subtract total costs from consumer payments to get profits
mrktsurplus_df = consumer_payments_df.copy()
mrktsurplus_df[['CEM', 'PF', 'DLAC', 'DLAC Imperfect', 'SLAC']] = consumer_payments_df[['CEM', 'PF', 'DLAC', 'DLAC Imperfect', 'SLAC']] - total_system_costs_df[['CEM', 'DLAC', 'DLAC Imperfect', 'SLAC']]

# Convert the profits to scientific notation
mrktsurplus_df[['CEM', 'PF', 'DLAC', 'DLAC Imperfect', 'SLAC']] = mrktsurplus_df[['CEM', 'PF', 'DLAC', 'DLAC Imperfect', 'SLAC']].applymap(lambda x: f"{x:.2e}")

# Save the profits dataframe to a CSV file
mrktsurplus_df.to_csv(tables_path + 'mrktsurplus_df.csv', index=False)

mrktsurplus_df

,Case Name,CEM,PF,DLAC,DLAC Imperfect,SLAC
0,Thermal_Base,0.00e+00,-4.53e+08,6.65e+09,nan,4.50e+09


In [210]:
# Create a new dataframe by dividing the 'DLAC', 'DLAC Imperfect', and 'SLAC' columns by the 'CEM' column
consumer_payments_ratio_df = consumer_payments_df.copy()
consumer_payments_ratio_df['CEM'] = consumer_payments_df['CEM'].astype(float) / consumer_payments_df['CEM'].astype(float) - 1
consumer_payments_ratio_df['PF'] = consumer_payments_df['PF'].astype(float) / consumer_payments_df['CEM'].astype(float) - 1
consumer_payments_ratio_df['DLAC'] = consumer_payments_df['DLAC'].astype(float) / consumer_payments_df['CEM'].astype(float) -1 
consumer_payments_ratio_df['DLAC Imperfect'] = consumer_payments_df['DLAC Imperfect'].astype(float) / consumer_payments_df['CEM'].astype(float) -1
consumer_payments_ratio_df['SLAC'] = consumer_payments_df['SLAC'].astype(float) / consumer_payments_df['CEM'].astype(float) -1

consumer_payments_ratio_df['CEM'] = consumer_payments_ratio_df['CEM'].round(2) * 100
consumer_payments_ratio_df['PF'] = consumer_payments_ratio_df['PF'].round(2) * 100
consumer_payments_ratio_df['DLAC'] = consumer_payments_ratio_df['DLAC'].round(2) * 100
consumer_payments_ratio_df['DLAC Imperfect'] = consumer_payments_ratio_df['DLAC Imperfect'].round(2) * 100
consumer_payments_ratio_df['SLAC'] = consumer_payments_ratio_df['SLAC'].round(2) * 100

# Remove the decimal point from the normalized total system costs
consumer_payments_ratio_df[['CEM', 'PF', 'DLAC', 'DLAC Imperfect', 'SLAC']] = \
    consumer_payments_ratio_df[['CEM', 'PF', 'DLAC', 'DLAC Imperfect', 'SLAC']].astype(int)

# Add a '%' symbol at the end of the normalized total system costs
consumer_payments_ratio_df[['CEM', 'PF', 'DLAC', 'DLAC Imperfect', 'SLAC']] = \
    consumer_payments_ratio_df[['CEM', 'PF', 'DLAC', 'DLAC Imperfect', 'SLAC']].astype(str) + '%'

consumer_payments_ratio_df.to_csv(tables_path + 'consumer_payments_ratio.csv', index=False)
consumer_payments_ratio_df

,Case Name,CEM,PF,DLAC,DLAC Imperfect,SLAC
0,Thermal_Base,0%,2%,-4%,2%,-10%


In [211]:
total_operating_costs_df = pd.DataFrame.from_dict(total_operating_costs_dic, orient='index').reset_index()
total_operating_costs_df.columns = ['Case Name', 'PF', 'CEM', 'DLAC', 'DLAC Imperfect', 'SLAC']
total_operating_costs_df.to_csv(tables_path + 'total_operating_costs.csv', index=False)
total_operating_costs_df

,Case Name,PF,CEM,DLAC,DLAC Imperfect,SLAC
0,Thermal_Base,1.0833e+10,1.1049e+10,1.0632e+10,4.5931e+09,4.5744e+09


In [212]:
from matplotlib.backends.backend_pdf import PdfPages

# BEGIN: Add unit profit tables to PDF

# create a date stamp of just the year, month and day
date = datetime.datetime.now()
date = date.strftime("%Y-%m-%d-%H-%M")

In [213]:
unit_profits_fln =  'unit_profits_' + date + '.pdf'

# Create a PDF file
with PdfPages(tables_path + unit_profits_fln) as pdf:
    for case_name in case_names:
        # Get the unit profit dataframe for the current case
        unit_profit_df = unit_profit_dic[case_name]

        # Create a figure and axis
        fig, ax = plt.subplots(figsize=(10, 6))

        # Hide the axes
        ax.xaxis.set_visible(False)
        ax.yaxis.set_visible(False)
        ax.set_frame_on(False)

        # Create a table
        table = ax.table(cellText=unit_profit_df.values, colLabels=unit_profit_df.columns, cellLoc='center', loc='center')

        # Set table style
        table.auto_set_font_size(False)
        table.set_fontsize(10)
        table.scale(1.2, 1.2)

        # Add a title
        ax.set_title(f'Unit Profits for {case_name} ($/MW)', fontsize=14, fontweight='bold')

        # Add the figure to the PDF
        pdf.savefig(fig)

        # Close the figure
        plt.close(fig)
# END: Add unit profit tables to PDF

In [214]:
percentage_profits_fln =  'percentage_profits_' + date + '.pdf'

# Create a PDF file
with PdfPages(tables_path + percentage_profits_fln) as pdf:
    for case_name in case_names:
        # Get the unit profit dataframe for the current case
        percentage_profits_df = percentage_profits_dic[case_name]

        # Create a figure and axis
        fig, ax = plt.subplots(figsize=(10, 6))

        # Hide the axes
        ax.xaxis.set_visible(False)
        ax.yaxis.set_visible(False)
        ax.set_frame_on(False)

        # Create a table
        table = ax.table(cellText=percentage_profits_df.values, \
                         colLabels=percentage_profits_df.columns, \
                            cellLoc='center', loc='center')

        # Set table style
        table.auto_set_font_size(False)
        table.set_fontsize(10)
        table.scale(1.2, 1.2)

        # Add a title
        ax.set_title(f'% Profits over Fixed Costs for {case_name}', fontsize=14, fontweight='bold')

        # Add the figure to the PDF
        pdf.savefig(fig)

        # Close the figure
        plt.close(fig)
# END: Add unit profit tables to PDF

In [215]:
# print 4 hour battery case percentage profits to csv
percentage_profits_dic['4_Hr_BESS'].to_csv(tables_path + '4_hour_battery_case_percentage_profits.csv', index=False)
percentage_profits_dic['4_Hr_BESS_Fuelx4'].to_csv(tables_path + '4_hour_battery_case_fuelx4_percentage_profits.csv', index=False)

KeyError: '4_Hr_BESS'

In [ ]:
battery_df = pd.DataFrame()
for key in percentage_profits_dic:
    print(f"Key: {key}")
    # print(percentage_profits_dic[key])
    battery_row = percentage_profits_dic[key][percentage_profits_dic[key]['Resource'] == 'Battery']
    battery_row.insert(0, 'Case', key)
    battery_df = pd.concat([battery_df, battery_row], ignore_index=True)
    if battery_df.empty:
        battery_df = None
    # print(battery_df)

battery_df.to_csv(tables_path + 'battery_percentage_profits.csv', index=False)

Key: Thermal_Base
Key: 2_Hr_BESS
Key: 2_Hr_BESS_Fuelx2
Key: 4_Hr_BESS
Key: 4_Hr_BESS_Fuelx2
Key: 4_Hr_BESS_Fuelx3
Key: 4_Hr_BESS_Fuelx4
Key: 6_Hr_BESS
Key: 6_Hr_BESS_Fuelx2


AttributeError: 'NoneType' object has no attribute 'to_csv'

In [ ]:
# # Convert the percentage profits for 'Thermal_Case' to a DataFrame
# thermal_case_df = percentage_profits_dic['Thermal_Case']

# # Define the filename
# thermal_case_csv_filename = pdf_path + 'thermal_case_percentage_profits.csv'

# # Save the DataFrame to a CSV file
# thermal_case_df.to_csv(thermal_case_csv_filename, index=False)

In [ ]:
consumer_payments_fln =  'consumer_payments_' + date + '.pdf'

# Create a PDF file
with PdfPages(tables_path + consumer_payments_fln) as pdf:
    # Create a figure and axis
    fig, ax = plt.subplots(figsize=(10, 6))

    # Hide the axes
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)
    ax.set_frame_on(False)

    # Create a table
    table = ax.table(cellText=consumer_payments_df.values, colLabels=consumer_payments_df.columns, cellLoc='center', loc='center')

    # Set table style
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1.2, 1.2)

    # Add a title
    ax.set_title('Consumer Payments for Each Case ($)', fontsize=14, fontweight='bold')

    # Add the figure to the PDF
    pdf.savefig(fig)

    # Close the figure
    plt.close(fig)

ErrorException: syntax: character literal contains multiple characters

In [ ]:
consumer_payments_ratio_fln =  'consumer_payments_ratio_' + date + '.pdf'

# Create a PDF file
with PdfPages(tables_path + consumer_payments_ratio_fln) as pdf:
    # Create a figure and axis
    fig, ax = plt.subplots(figsize=(10, 6))

    # Hide the axes
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)
    ax.set_frame_on(False)

    # Create a table
    table = ax.table(cellText=consumer_payments_ratio_df.values, colLabels=consumer_payments_ratio_df.columns, cellLoc='center', loc='center')

    # Set table style
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1.2, 1.2)

    # Add a title
    ax.set_title('Ratio of Consumer Payments (Normalized by CEM)', fontsize=14, fontweight='bold')

    # Add the figure to the PDF
    pdf.savefig(fig)

    # Close the figure
    plt.close(fig)

ErrorException: syntax: character literal contains multiple characters

In [ ]:
pmr_cpr_dic = {}

for case_name in case_names:
    percentage_profits_df = percentage_profits_dic[case_name]
    consumer_payments_ratio = consumer_payments_ratio_df[consumer_payments_ratio_df['Case Name'] == case_name].iloc[0]
    
    # Create a new row with the consumer payments ratio
    new_row = pd.DataFrame({
        'Resource': ['Consumer Payments Ratio'],
        'CEM': [f"{float(consumer_payments_ratio['CEM']) * 100:.2f}%"],
        'DLAC': [f"{float(consumer_payments_ratio['DLAC']) * 100:.2f}%"],
        'DLAC Imperfect': [f"{float(consumer_payments_ratio['DLAC Imperfect']) * 100:.2f}%"],
        'SLAC': [f"{float(consumer_payments_ratio['SLAC']) * 100:.2f}%"]
    })
    
    # Append the new row to the dataframe
    percentage_profits_df = pd.concat([percentage_profits_df, new_row], ignore_index=True)
    
    # Update the dictionary
    pmr_cpr_dic[case_name] = percentage_profits_df

# pmr_cpr_dic
# concatenate all the dataframes in pmr_cpr_dic, adding a row for the name of the case
pmr_cpr_combined_df = pd.DataFrame()

for case_name, df in pmr_cpr_dic.items():
    df.insert(0, 'Case Name', case_name)
    pmr_cpr_combined_df = pd.concat([pmr_cpr_combined_df, df], ignore_index=True)


# replace the names of the cases with the print names
pmr_cpr_combined_df['Case Name'] = pmr_cpr_combined_df['Case Name'].replace('2_Hour_Batteries_Case', '2 Hr Bat')
pmr_cpr_combined_df['Case Name'] = pmr_cpr_combined_df['Case Name'].replace('2_Hour_Batteries_Case_Fuelx2', '2 Hr Bat, Fuel Costs x2')
pmr_cpr_combined_df['Case Name'] = pmr_cpr_combined_df['Case Name'].replace('4_Hour_Batteries_Case', '4 Hr Bat')
pmr_cpr_combined_df['Case Name'] = pmr_cpr_combined_df['Case Name'].replace('4_Hour_Batteries_Case_Fuelx2', '4 Hr Bat, Fuel Costs x2')
pmr_cpr_combined_df['Case Name'] = pmr_cpr_combined_df['Case Name'].replace('4_Hour_Batteries_Case_Fuelx3', '4 Hr Bat, Fuel Costs x3')
pmr_cpr_combined_df['Case Name'] = pmr_cpr_combined_df['Case Name'].replace('4_Hour_Batteries_Case_Fuelx4', '4 Hr Bat, Fuel Costs x4')
pmr_cpr_combined_df['Case Name'] = pmr_cpr_combined_df['Case Name'].replace('6_Hour_Batteries_Case', '6 Hour Bat')
pmr_cpr_combined_df['Case Name'] = pmr_cpr_combined_df['Case Name'].replace('6_Hour_Batteries_Case_Fuelx2', '6 Hr Bat, Fuel Costs x2')
pmr_cpr_combined_df['Case Name'] = pmr_cpr_combined_df['Case Name'].replace('8_Hour_Batteries_Case', '8 Hr Bat')
pmr_cpr_combined_df['Case Name'] = pmr_cpr_combined_df['Case Name'].replace('8_Hour_Batteries_Case_Fuelx2', '8 Hr Bat, Fuel Costs x2')
pmr_cpr_combined_df['Case Name'] = pmr_cpr_combined_df['Case Name'].replace('10_Hour_Batteries_Case', '10 Hr Bat')
pmr_cpr_combined_df['Case Name'] = pmr_cpr_combined_df['Case Name'].replace('10_Hour_Batteries_Case_Fuelx2', '10 Hr Bat, Fuel Costs x2')
pmr_cpr_combined_df['Case Name'] = pmr_cpr_combined_df['Case Name'].replace('Thermal_Case', 'Thermal')

pmr_cpr_combined_df.to_csv(tables_path + 'pmr_cpr_combined_' + date + '.csv', index=False)

ErrorException: syntax: { } vector syntax is discontinued around c:\Users\ks885\Documents\aa_research\Modeling\spcm_genx_experiment\figures\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y111sZmlsZQ==.jl:1

In [ ]:

total_system_costs_fln =  'total_system_costs_' + date + '.pdf'

# Create a PDF file
with PdfPages(tables_path + total_system_costs_fln) as pdf:

    # Create a figure and axis
    fig, ax = plt.subplots(figsize=(10, 6))

    # Hide the axes
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)
    ax.set_frame_on(False)

    # Create a table
    table = ax.table(cellText=total_system_costs_df.values, colLabels=total_system_costs_df.columns, cellLoc='center', loc='center')

    # Set table style
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1.2, 1.2)

    # Add a title
    ax.set_title(f'Total Costs for each case ($)', fontsize=14, fontweight='bold')

    # Add the figure to the PDF
    pdf.savefig(fig)

    # Close the figure
    plt.close(fig)
# END: Add unit profit tables to PDF

ErrorException: syntax: character literal contains multiple characters

In [ ]:
total_nse_df.columns

UndefVarError: UndefVarError: `total_nse_df` not defined

In [ ]:
total_nse_fln =  'total_nse_' + date + '.pdf'

# Create a PDF file
with PdfPages(tables_path + total_nse_fln) as pdf:
    # Create a figure and axis
    fig, ax = plt.subplots(figsize=(10, 6))

    # Hide the axes
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)
    ax.set_frame_on(False)

    # Create a table
    table = ax.table(cellText=total_nse_df.values, colLabels=total_nse_df.columns, cellLoc='center', loc='center')

    # Set table style
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1.2, 1.2)

    # Add a title
    ax.set_title('Total Non-served Energy for Each Case (MW)', fontsize=14, fontweight='bold')

    # Add the figure to the PDF
    pdf.savefig(fig)

    # Close the figure
    plt.close(fig)

ErrorException: syntax: character literal contains multiple characters

In [ ]:
total_unmet_rsv_fln =  'total_unmet_rsv_' + date + '.pdf'

# Create a PDF file
with PdfPages(tables_path + total_unmet_rsv_fln) as pdf:
    # Create a figure and axis
    fig, ax = plt.subplots(figsize=(10, 6))

    # Hide the axes
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)
    ax.set_frame_on(False)

    # Create a table
    table = ax.table(cellText=total_unmet_rsv_df.values, colLabels=total_unmet_rsv_df.columns, cellLoc='center', loc='center')

    # Set table style
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1.2, 1.2)

    # Add a title
    ax.set_title('Total Unmet Reserves for Each Case (MW)', fontsize=14, fontweight='bold')

    # Add the figure to the PDF
    pdf.savefig(fig)

    # Close the figure
    plt.close(fig)

ErrorException: syntax: character literal contains multiple characters